In [2]:
import os
import sys
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import math
import statsmodels.api as sm
from patsy import dmatrices
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.preprocessing import FunctionTransformer, PolynomialFeatures
from sklearn.decomposition import FactorAnalysis, PCA
from sklearn.feature_selection import GenericUnivariateSelect, RFE
from sklearn import metrics
from sklearn import linear_model, decomposition, datasets
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import f_regression, mutual_info_regression
import seaborn as sns # More snazzy plotting library
import itertools
from itertools import  product
import pprint
from sklearn.ensemble import ExtraTreesRegressor
from multiprocessing import Process, Value, Array
#from asyncio import Queue
from threading import Thread
import pickle
import shutil
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.metrics import r2_score

path_full = "../dataset/no_imdb_names-count_cat-tf_184f.csv"
path_train = "../dataset/no_imdb_names-count_cat-tf_184f_train.csv"
path_test = "../dataset/no_imdb_names-count_cat-tf_184f_test.csv"

dta_full = pd.read_csv(path_full)
dta_full = dta_full.fillna(value=0, axis=1)
dta_full = dta_full.dropna()
dta_full = dta_full.drop('Unnamed: 0', axis=1)

dta_train = pd.read_csv(path_train)
dta_train = dta_train.fillna(value=0, axis=1)
dta_train = dta_train.dropna()
dta_train = dta_train.drop('Unnamed: 0', axis=1)

dta_test = pd.read_csv(path_test)
dta_test = dta_test.fillna(value=0, axis=1)
dta_test = dta_test.dropna()
dta_test = dta_test.drop('Unnamed: 0', axis=1)

def label_gross_2 (gross):
    if (gross < 200000000) : return 1
    elif (gross >= 200000000) : return 2

def label_gross_3 (gross):
    if (gross < 10000000) : return 1
    elif ((gross >= 10000000) & (gross < 300000000)) : return 2
    elif (gross >= 300000000) : return 3

def label_gross_4 (gross):
    if (gross < 5000000) : return 1
    elif ((gross >= 5000000) & (gross < 50000000)) : return 2
    elif ((gross >= 50000000) & (gross < 350000000)) : return 3
    elif (gross >= 350000000) : return 4

def label_gross_5 (gross):
    if (gross < 1000000) : return 1
    elif ((gross >= 1000000) & (gross < 25000000)) : return 2
    elif ((gross >= 25000000) & (gross < 100000000)) : return 3
    elif ((gross >= 100000000) & (gross < 400000000)) : return 4
    elif (gross >= 400000000) : return 5
    
def label_gross_6 (gross):
    if (gross < 1000000) : return 1
    elif ((gross >= 1000000) & (gross < 25000000)) : return 2
    elif ((gross >= 25000000) & (gross < 50000000)) : return 3
    elif ((gross >= 50000000) & (gross < 150000000)) : return 4
    elif ((gross >= 150000000) & (gross < 450000000)) : return 5
    elif (gross >= 450000000) : return 6

def label_gross_7 (gross):
    if (gross < 500000) : return 1
    elif ((gross >= 500000) & (gross < 5000000)) : return 2
    elif ((gross >= 5000000) & (gross < 50000000)) : return 3
    elif ((gross >= 50000000) & (gross < 150000000)) : return 4
    elif ((gross >= 150000000) & (gross < 200000000)) : return 5
    elif ((gross >= 200000000) & (gross < 500000000)) : return 6
    elif (gross >= 500000000) : return 7
    
def label_gross_8 (gross):
    if (gross < 500000) : return 1
    elif ((gross >= 500000) & (gross < 5000000)) : return 2
    elif ((gross >= 5000000) & (gross < 20000000)) : return 3
    elif ((gross >= 20000000) & (gross < 50000000)) : return 4
    elif ((gross >= 50000000) & (gross < 100000000)) : return 5
    elif ((gross >= 100000000) & (gross < 250000000)) : return 6
    elif ((gross >= 250000000) & (gross < 550000000)) : return 7
    elif (gross >= 550000000) : return 8

def label_gross_9 (gross):
    if (gross < 500000) : return 1
    elif ((gross >= 500000) & (gross < 5000000)) : return 2
    elif ((gross >= 5000000) & (gross < 20000000)) : return 3
    elif ((gross >= 20000000) & (gross < 50000000)) : return 4
    elif ((gross >= 50000000) & (gross < 70000000)) : return 5
    elif ((gross >= 70000000) & (gross < 125000000)) : return 6
    elif ((gross >= 125000000) & (gross < 250000000)) : return 7
    elif ((gross >= 250000000) & (gross < 550000000)) : return 8
    elif (gross >= 550000000) : return 9
    
def label_gross_10 (gross):
    if    (gross  < 500000) : return 1
    elif ((gross >= 500000)    & (gross < 5000000)) : return 2
    elif ((gross >= 5000000)   & (gross < 20000000)) : return 3
    elif ((gross >= 20000000)  & (gross < 50000000)) : return 4
    elif ((gross >= 50000000)  & (gross < 70000000)) : return 5
    elif ((gross >= 70000000)  & (gross < 125000000)) : return 6
    elif ((gross >= 125000000) & (gross < 250000000)) : return 7
    elif ((gross >= 250000000) & (gross < 400000000)) : return 8
    elif ((gross >= 400000000) & (gross < 600000000)) : return 9
    elif  (gross >= 600000000) : return 10
    

##change power here  
pw = 4
def log_poly(X):
    #hardcoe for this test
    global pw
    #do log
    df_t = pd.DataFrame(X[:,:10])
    X_t = df_t.replace(0, 1/math.e)
    log_res = np.log(X_t)
    
    #do poly
    vector = X[:,10:]
    res    = X[:,:10]
    X      = X[:,:10]
    for power in range(2,pw + 1):
        res = np.concatenate((res, np.power(X, power)), axis=1)
    res_poly_log = np.concatenate((res, log_res), axis=1)
    
    #return conat results
    return np.concatenate((res_poly_log, vector), axis=1)



def rename_log_poly(X):
    #hardcoe for this test
    global pw
    #do log
    tran_f = X[:10]
    print(len(tran_f))
    #X_t = df_t.replace(0, 1/math.e)
    log_res = list(map(lambda x: "log(" + x + ")" , tran_f))
    
    #do poly
    vector = X[10:]
    res    = X[:10]
    X      = X[:10]
    for power in range(2,pw + 1):
        res = np.concatenate((res, list(map(lambda x: x + "^" + str(power) , X))), axis=1)
    res_poly_log = np.concatenate((res, log_res), axis=1)
    
    #return conat results
    return np.concatenate((res_poly_log, vector), axis=1)


LogPolynomialTransformer = FunctionTransformer(log_poly)

preprocessor = LogPolynomialTransformer

transfomer = StandardScaler()

reducer = RFE(ExtraTreesRegressor(), step=0.1, n_features_to_select =round((dta_train.shape[1] + (pw -1) *10)*0.33) )


pipe_cl = Pipeline(steps=[('preprocessor', preprocessor), ('transfomer', transfomer), ('reducer', reducer)])
pipe_reg = Pipeline(steps=[('preprocessor', preprocessor), ('transfomer', transfomer), ('reducer', reducer)])

x_train = dta_train.drop('worldwide_gross', axis=1)
y_train_cl = dta_train.worldwide_gross.apply (lambda gross: label_gross_3 (gross))
y_train_reg = dta_train['worldwide_gross']

x_test = dta_test.drop('worldwide_gross', axis=1)
y_test_cl = dta_test.worldwide_gross.apply (lambda gross: label_gross_3 (gross))
y_test_reg = dta_test['worldwide_gross']

x_train_labels = rename_log_poly(list(dta_train.drop('worldwide_gross', axis=1).columns.values))

10


In [3]:
pipe_cl.fit_transform(x_train,y_train_cl)
pipe_cl.fit_transform(x_train,y_train_reg)

selected_label_cl = [x_train_labels[ind] for ind, val in enumerate(pipe_cl.named_steps['reducer'].ranking_) if val ==1]
selected_label_reg = [x_train_labels[ind] for ind, val in enumerate(pipe_reg.named_steps['reducer'].ranking_) if val ==1]

In [4]:
selected_label_cl

['actor_1_facebook_likes',
 'actor_2_facebook_likes',
 'actor_3_facebook_likes',
 'cast_total_facebook_likes',
 'director_facebook_likes',
 'duration',
 'production_budget',
 'title_year',
 'actor_1_facebook_likes^2',
 'actor_2_facebook_likes^2',
 'actor_3_facebook_likes^2',
 'blockbuster_month^2',
 'cast_total_facebook_likes^2',
 'director_facebook_likes^2',
 'dump_month^2',
 'duration^2',
 'production_budget^2',
 'title_year^2',
 'actor_1_facebook_likes^3',
 'actor_2_facebook_likes^3',
 'actor_3_facebook_likes^3',
 'blockbuster_month^3',
 'cast_total_facebook_likes^3',
 'director_facebook_likes^3',
 'duration^3',
 'production_budget^3',
 'title_year^3',
 'actor_1_facebook_likes^4',
 'actor_2_facebook_likes^4',
 'actor_3_facebook_likes^4',
 'cast_total_facebook_likes^4',
 'director_facebook_likes^4',
 'duration^4',
 'production_budget^4',
 'title_year^4',
 'log(actor_1_facebook_likes)',
 'log(actor_2_facebook_likes)',
 'log(actor_3_facebook_likes)',
 'log(blockbuster_month)',
 'log(ca

In [5]:
selected_label_reg

['actor_1_facebook_likes',
 'actor_2_facebook_likes',
 'actor_3_facebook_likes',
 'cast_total_facebook_likes',
 'director_facebook_likes',
 'duration',
 'production_budget',
 'title_year',
 'actor_1_facebook_likes^2',
 'actor_2_facebook_likes^2',
 'actor_3_facebook_likes^2',
 'blockbuster_month^2',
 'cast_total_facebook_likes^2',
 'director_facebook_likes^2',
 'dump_month^2',
 'duration^2',
 'production_budget^2',
 'title_year^2',
 'actor_1_facebook_likes^3',
 'actor_2_facebook_likes^3',
 'actor_3_facebook_likes^3',
 'blockbuster_month^3',
 'cast_total_facebook_likes^3',
 'director_facebook_likes^3',
 'duration^3',
 'production_budget^3',
 'title_year^3',
 'actor_1_facebook_likes^4',
 'actor_2_facebook_likes^4',
 'actor_3_facebook_likes^4',
 'cast_total_facebook_likes^4',
 'director_facebook_likes^4',
 'duration^4',
 'production_budget^4',
 'title_year^4',
 'log(actor_1_facebook_likes)',
 'log(actor_2_facebook_likes)',
 'log(actor_3_facebook_likes)',
 'log(blockbuster_month)',
 'log(ca